Fetch Data

In [2]:
import requests
import pandas as pd
from io import StringIO

# Function to download and read CSV file into a DataFrame
def read_csv_from_github(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Read CSV data into a DataFrame
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data)
        return df
    else:
        print(f"Failed to download CSV file from {url}")
        return None

# GitHub repository URL and folder path
repo_url = "https://api.github.com/repos/euanwm/OpenWeightlifting/contents/backend/event_data/IWF"

# Get list of files in the folder from GitHub API
response = requests.get(repo_url)
if response.status_code == 200:
    files = response.json()
    # List to store DataFrames
    dataframes = []
    # Iterate over each file in the folder
    for file in files:
        if file["type"] == "file" and file["name"].endswith(".csv"):
            # Construct URL for raw CSV file
            csv_url = file["download_url"]
            # Download and read CSV file into a DataFrame
            df = read_csv_from_github(csv_url)
            if df is not None:
                dataframes.append(df)
else:
    print(f"Failed to get list of files from {repo_url}")

Data Wrangling 

In [36]:
# Combine Data
IWF_data = pd.concat(dataframes, ignore_index=True)

# Data Cleaning
IWF_data['sex'] = IWF_data['category'].str.split().str[-1]
IWF_data['date'] = pd.to_datetime(IWF_data['date'])
IWF_data['category'] = IWF_data['category'].replace({
'49 kg Men': '01. 55 kg',
'55 kg Men': '01. 55 kg',
'61 kg Men': '02. 61 kg',
'67 kg Men': '03. 67 kg',
'73 kg Men': '04. 73 kg',
'81 kg Men': '05. 81 kg',
'89 kg Men': '06. 89 kg',
'96 kg Men': '07. 96 kg',
'102 kg Men': '08. 102 kg',
'109 kg Men': '09. 109 kg',
'P109 kg Men': '09. 109 kg',
'+102 kg Men': '10. +109 kg',
'+109 kg Men': '10. +109 kg',
'+81 kg Men': '10. +109 kg',
'+89 kg Men': '10. +109 kg',
'45 kg Women': '01. 45 kg',
'40 kg Women': '01. 45 kg',
'49 kg Women': '02. 49 kg',
'55 kg Women': '03. 55 kg',
'59 kg Women': '04. 59 kg',
'64 kg Women': '05. 64 kg',
'71 kg Women': '06. 71 kg',
'76 kg Women': '07. 76 kg',
'81 kg Women': '08. 81 kg',
'87 kg Women': '09. 87 kg',
'+87 kg Women': '10. +87 kg',
'+81 kg Women': '10. +87 kg',
'+64 kg Women': '10. +87 kg',
'+71 kg Women': '10. +87 kg'
})

# Filter rows
IWF_data = IWF_data[IWF_data['date'].dt.year >= 2022]

# Wrangle Data
# Snatches
snatch = IWF_data.groupby(['lifter_name', 'category', 'sex'])['best_snatch'].max().reset_index()
snatch = snatch.rename(columns={'best_snatch': 'weight (kg)'})
snatch = snatch.sort_values(by=['category', 'weight (kg)'], ascending=[True, False])
snatch['rank'] = snatch.groupby('category').cumcount() + 1
snatch = snatch[snatch['rank'] <= 10]
snatch_w = snatch[snatch['sex'] == 'Women']
snatch_m = snatch[snatch['sex'] == 'Men']

# Clean and Jerks
CandJ = IWF_data.groupby(['lifter_name', 'category', 'sex'])['best_cj'].max().reset_index()
CandJ = CandJ.rename(columns={'best_cj': 'weight (kg)'})
CandJ = CandJ.sort_values(by=['category', 'weight (kg)'], ascending=[True, False])
CandJ['rank'] = CandJ.groupby('category').cumcount() + 1
CandJ = CandJ[CandJ['rank'] <= 10]
CandJ_w = CandJ[CandJ['sex'] == 'Women']
CandJ_m = CandJ[CandJ['sex'] == 'Men']

# Total
total = IWF_data.groupby(['lifter_name', 'category', 'sex'])['total'].max().reset_index()
total = total.rename(columns={'total': 'weight (kg)'})
total = total.sort_values(by=['category', 'weight (kg)'], ascending=[True, False])
total['rank'] = total.groupby('category').cumcount() + 1
total = total[total['rank'] <= 10]
total_w = total[total['sex'] == 'Women']
total_m = total[total['sex'] == 'Men']

print(snatch_w, snatch_m, CandJ_w, CandJ_m, total_w, total_m)


                       lifter_name    category    sex  weight (kg)  rank
4638                  WON Hyon Sim   01. 45 kg  Women         87.0     1
4176         SUKCHAROEN Thanyathon   01. 45 kg  Women         82.0     2
2174               KHONG My Phuong   01. 45 kg  Women         80.0     3
3019               MY PHUONG Khong   01. 45 kg  Women         78.0     4
3459    PRAMONGKHOL Miss Siriwimon   01. 45 kg  Women         78.0     5
...                            ...         ...    ...          ...   ...
10                    ABBAS Halima  10. +87 kg  Women        124.0     6
785                 CAMPBELL Emily  10. +87 kg  Women        124.0     7
11    ABBAS Halima Abdelazim Sedky  10. +87 kg  Women        122.0     8
786            CAMPBELL Emily Jade  10. +87 kg  Women        122.0     9
4153              STOWERS Feagaiga  10. +87 kg  Women        121.0    10

[100 rows x 5 columns]                  lifter_name     category  sex  weight (kg)  rank
2346           LAI Gia Thanh    01

Data Visualisation

In [96]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_box_with_top_lifter(df, title, ticks):

    # create dataframe without top 3 lifters 
    
    df2 = df[df['rank'] >= 4]
    rows_to_duplicate = df[df['rank'] == 4]
    df2 = df2.append(rows_to_duplicate, ignore_index=True)
    df2 = df2.append(rows_to_duplicate, ignore_index=True)
    df2 = df2.append(rows_to_duplicate, ignore_index=True)
    
    # Set the style of seaborn
    sns.set(style="whitegrid")

    # Create a box plot for each weight category
    min_weight = np.floor(df['weight (kg)'].min() / ticks) * ticks  # Round down to nearest multiple of 5
    max_weight = np.ceil(df['weight (kg)'].max() / ticks) * ticks  # Round up to nearest multiple of 5
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='category', y='weight (kg)', data=df2)
    plt.title(title)
    plt.xlabel('Weight Category')
    plt.ylabel('Weight Lifted (kg)')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.ylim(min_weight - ticks/2, max_weight + 3 * ticks)
    
    # Customize y-axis ticks and grid lines
    plt.yticks(np.arange(min_weight, max_weight + 1, ticks))  # Set y-axis ticks every 5 kg
    plt.grid(axis='y', linestyle='-', linewidth=0.5, color='grey')  # Set grid lines
    
    # Annotate the plot with the names of top lifters
    rank_colors = {1: 'gold', 2: 'silver', 3: '#cd7f32'}
    vertical_offsets = {1: ticks * 2, 2: ticks * 3/2, 3: ticks}
    horizontal_offsets = {1: -0.1, 2: 0.0, 3: 0.1}
    for rank in (3, 2, 1):  # Iterate over ranks 1, 2, and 3
        lifter = df[df['rank'] == rank]
        for i in range(len(lifter)):
            plt.text(i, max_weight + vertical_offsets[rank], lifter['lifter_name'].iloc[i], rotation=30,
                    horizontalalignment='center', verticalalignment='center', fontsize=7, color=rank_colors[rank])
        for i, row in lifter.iterrows():
            category_index = df['category'].unique().tolist().index(row['category'])  # Get the numerical index of the category
            plt.scatter(category_index + horizontal_offsets[rank], row['weight (kg)'], color=rank_colors[rank], marker='o', s=64, zorder=5)
    
    plt.tight_layout()  # Adjust layout to prevent overlap
        # Call plt.savefig() to save the plot to a file
    plt.savefig(title, dpi=300)  # Set dpi (dots per inch) for higher resolution if needed
    plt.close()  # Close the figure to release memory

# Call the function for each DataFrame
plot_box_with_top_lifter(snatch_w, 'Womens Top 10 Snatch in IWF competitions since 2022 by weight category.', 5)
plot_box_with_top_lifter(CandJ_w, 'Womens Top 10 Clean and Jerk in IWF competitions since 2022 by weight category.', 5)
plot_box_with_top_lifter(total_w, 'Womens Top 10 Total in IWF competitions since 2022 by weight category.', 10)
plot_box_with_top_lifter(snatch_m, 'Mens Top 10 Snatch in IWF competitions since 2022 by weight category.', 5)
plot_box_with_top_lifter(CandJ_m, 'Mens Top 10 Clean and Jerk in IWF competitions since 2022 by weight category.', 5)
plot_box_with_top_lifter(total_m, 'Mens Top 10 total in IWF competitions since 2022 by weight category.', 10)


C:\Users\jkman\AppData\Local\Temp\ipykernel_24220\2647652230.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(rows_to_duplicate, ignore_index=True)
C:\Users\jkman\AppData\Local\Temp\ipykernel_24220\2647652230.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(rows_to_duplicate, ignore_index=True)
C:\Users\jkman\AppData\Local\Temp\ipykernel_24220\2647652230.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(rows_to_duplicate, ignore_index=True)
C:\Users\jkman\AppData\Local\Temp\ipykernel_24220\2647652230.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(rows_t